In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stumbleupon/test.tsv
/kaggle/input/stumbleupon/train.tsv
/kaggle/input/stumbleupon/sampleSubmission.csv
/kaggle/input/stumbleupon/raw_content.zip


In [2]:
import matplotlib.pyplot as plt 
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from nltk.stem import PorterStemmer
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import plotly.express as px
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# taking all columns of training set only for data exploration
df_train=pd.read_csv('../input/stumbleupon/train.tsv',sep='\t')

# taking boilerplate column as an input for the model beacuse only this column contain lot of high quality text data useful for our nlp task
df_test=pd.read_csv('../input/stumbleupon/test.tsv',sep='\t')

In [ ]:
df_test.head()

In [ ]:
df_train.columns

In [ ]:
df_test.shape

In [4]:
df_train=df_train.dropna()

In [5]:
df_train.reset_index(inplace=True)
df_train.shape

(7395, 28)

In [ ]:
df_train['boilerplate'][0]

In [ ]:
df_train.columns

In [ ]:
varValue = df_train.label.value_counts()
print(varValue)

In [ ]:
df_train['label'].value_counts().plot(kind='pie',colors=['#2C4373', '#F2A74B'],autopct='%1.1f%%',figsize=(9,9))
plt.show

In [ ]:
df_train['alchemy_category'].value_counts()

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(x=df_train['alchemy_category'],hue=df_train['label']);
plt.xlabel('Category');
plt.xticks(rotation=90);

In [ ]:
x_data=df_train.drop(['url', 'urlid', 'boilerplate','alchemy_category','framebased','hasDomainLink','is_news','lengthyLinkDomain','news_front_page'],axis=1)

X = x_data.drop("label",1)
y = x_data["label"]
x_data.head()
plt.figure(figsize=(15,7))
cor = x_data.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

cor_target = abs(cor["label"]) #absolute value

In [ ]:
relevant_features = cor_target[cor_target>=0.1]

In [6]:
import json

title=[]
body=[]
label=[]
alchemy_category_score=[]
urlid=[]
url=[]
linkwordscore=[]
frameTagRatio =[]
commonlinkratio_3    =[]
for i in range(7395):
    X=json.loads(df_train["boilerplate"][i])
    if 'title' in X.keys() and 'body' in X.keys() and 'url' in X.keys():
        title.append(X.get('title'))
        body.append(X.get('body'))
        url.append(X.get('url'))
        label.append(df_train['label'][i])
        alchemy_category_score.append(df_train['alchemy_category_score'][i])
        urlid.append(df_train['urlid'][i])
        linkwordscore.append(df_train['linkwordscore'][i])
        frameTagRatio.append(df_train['frameTagRatio'][i])
        commonlinkratio_3.append(df_train['commonlinkratio_3'][i])
    

dict_df = {'urlid':urlid,'title':title,'body':body,'url':url,'linkwordscore':linkwordscore,'frameTagRatio':frameTagRatio,'alchemy_category_score':alchemy_category_score,'commonlinkratio_3':commonlinkratio_3,'label':label}
data_df = pd.DataFrame.from_dict(dict_df)



In [ ]:
data_df.columns

In [9]:
data_df['body'].isnull().value_counts()

False    7278
True       57
Name: body, dtype: int64

In [7]:
data_df=data_df.dropna()
data_df.reset_index(inplace=True,drop=True)

In [11]:
data_df['body'].isnull().value_counts()

False    7267
Name: body, dtype: int64

In [8]:
import json

title=[]
body=[]
alchemy_category_score=[]
urlid=[]
url=[]
linkwordscore=[]
frameTagRatio =[]
commonlinkratio_3    =[]

for i in range(len(df_test)):
    X=json.loads(df_test["boilerplate"][i])
    if 'title' in X.keys() and 'body' in X.keys() and 'url'in X.keys():
        title.append(X.get('title'))
        body.append(X.get('body'))
        url.append(X.get('url'))
        alchemy_category_score.append(df_test['alchemy_category_score'][i])
        urlid.append(df_test['urlid'][i])
        linkwordscore.append(df_test['linkwordscore'][i])
        frameTagRatio.append(df_test['frameTagRatio'][i])
        commonlinkratio_3.append(df_test['commonlinkratio_3'][i])

dict_df = {'urlid':urlid,'title':title,'body':body,'url':url,'linkwordscore':linkwordscore,'frameTagRatio':frameTagRatio,'alchemy_category_score':alchemy_category_score,'commonlinkratio_3':commonlinkratio_3}
test_df = pd.DataFrame.from_dict(dict_df)

test_df=test_df.dropna()
test_df.reset_index(inplace=True)

In [ ]:
test_df.shape

In [9]:
X=data_df.drop('label',axis=1).values### independent features
y=data_df['label'].values###dependent features

In [ ]:
data_df.columns

In [ ]:
data_df['alchemy_category_score'].dtype

In [10]:

import nltk
import re
from nltk.corpus import stopwords

nltk.download('stopwords')



[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(data_df)):
    
    content = re.sub('[^a-zA-Z]', ' ', data_df['body'][i])
    content = content.lower()
    content = content.split()
    
    content = [ps.stem(word) for word in content if not word in stopwords.words('english')]
    content = ' '.join(content)
    corpus.append(content)
print('end')

end


In [ ]:
data_df['title'][0]

In [ ]:
len(corpus)

In [12]:
vocabulary = set()
for corpu in corpus:
        if corpu not in vocabulary:
            vocabulary.add(corpu)

In [13]:
vocab_length = len(vocabulary)

In [14]:
max_seq_length = 0

for title in corpus:
    if len(title) > max_seq_length:
        max_seq_length = len(title)


In [15]:
print("Vocab length:", vocab_length)
print("Max sequence length:", max_seq_length)

Vocab length: 6539
Max sequence length: 21816


In [16]:
onehot_repr=[one_hot(words,vocab_length)for words in corpus] 

In [17]:

embedded_docs = pad_sequences(onehot_repr, maxlen=300)

In [ ]:
embedded_docs[0]

In [18]:
for i in range(0,data_df.shape[0]):
    if(data_df['alchemy_category_score'][i] == '?'):
        data_df['alchemy_category_score'][i]= '0'
        
data_df['alchemy_category_score'] = data_df['alchemy_category_score'].astype(float)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
mean1=0;
mean2=0;
c1=0;
c2=0;
for i in range(0,len(data_df['alchemy_category_score'])):
    if(data_df['label'][i]==0):
        mean1=mean1+float(data_df['alchemy_category_score'][i])
        c1=c1+1
    if(data_df['label'][i]==1):
        mean2=mean2+float(data_df['alchemy_category_score'][i])
        c2=c2+1
mean2=mean2/c2
mean1=mean1/c1
    

In [20]:
for i in range(0,len(data_df['alchemy_category_score'])):
    if(data_df['alchemy_category_score'][i]==0 &data_df['label'][i]==0):
        data_df['alchemy_category_score'][i]=mean1
    if(data_df['alchemy_category_score'][i]==0 &data_df['label'][i]==1):
         data_df['alchemy_category_score'][i]=mean2

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
data_df.columns

In [21]:
temp1 = np.array(data_df['alchemy_category_score'].values.tolist())
print("shape before = ",temp1.shape)
temp1 = np.reshape(temp1,(-1,1))
print("shape after = ",temp1.shape)

temp2= np.array(data_df['linkwordscore'].values.tolist())
temp2 = np.reshape(temp2,(-1,1))

temp3= np.array(data_df['frameTagRatio'].values.tolist())
temp3 = np.reshape(temp3,(-1,1))


temp4= np.array(data_df['commonlinkratio_3'].values.tolist())
temp4 = np.reshape(temp4,(-1,1))


shape before =  (7267,)
shape after =  (7267, 1)


In [ ]:
y_final.shape

In [3]:

seq_len = 300  

z_final=temp2

y_final=np.array(y)
X_train,X_test,y_train,y_test,z_train,z_test=train_test_split(embedded_docs,y_final,z_final,test_size=0.25,random_state=0)

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

train_data = TensorDataset(torch.from_numpy(X_train),torch.from_numpy(z_train), torch.from_numpy(y_train))
val_data = TensorDataset(torch.from_numpy(X_test),torch.from_numpy(z_test), torch.from_numpy(y_test))

batch_size = 300

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)

NameError: name 'temp2' is not defined

In [23]:
class lstm_model(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(lstm_model, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x,f2,hidden):
        batch_size = x.size(0)
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds,hidden)
     
        out=torch.cat([lstm_out,f2])
        out = self.dropout(lstm_out)
        
        out = self.fc(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

In [2]:
vocab_size = vocab_length + 1
output_size = 1
embedding_dim = 300
hidden_dim = 512
n_layers = 2

model = lstm_model(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
lr=0.005
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

NameError: name 'vocab_length' is not defined

In [34]:
 for inputs1,inputs2, labels in train_loader:
        print(inputs1.shape)
        print(inputs2.shape)
        print(labels.shape)

torch.Size([400, 300])
torch.Size([400, 1])
torch.Size([400])
torch.Size([400, 300])
torch.Size([400, 1])
torch.Size([400])
torch.Size([400, 300])
torch.Size([400, 1])
torch.Size([400])
torch.Size([400, 300])
torch.Size([400, 1])
torch.Size([400])
torch.Size([400, 300])
torch.Size([400, 1])
torch.Size([400])
torch.Size([400, 300])
torch.Size([400, 1])
torch.Size([400])
torch.Size([400, 300])
torch.Size([400, 1])
torch.Size([400])
torch.Size([400, 300])
torch.Size([400, 1])
torch.Size([400])
torch.Size([400, 300])
torch.Size([400, 1])
torch.Size([400])
torch.Size([400, 300])
torch.Size([400, 1])
torch.Size([400])
torch.Size([400, 300])
torch.Size([400, 1])
torch.Size([400])
torch.Size([400, 300])
torch.Size([400, 1])
torch.Size([400])
torch.Size([400, 300])
torch.Size([400, 1])
torch.Size([400])
torch.Size([250, 300])
torch.Size([250, 1])
torch.Size([250])


In [25]:

from sklearn.metrics import accuracy_score
import time
start_time=time.time()
epochs=1000
final_losses=[]
for i in range(epochs):
    i=i+1
    for i in range(epochs):
        hidden = model.init_hidden(batch_size)
    
        for inputs1,inputs2, labels in train_loader:
            y_pred=model.forward(inputs1,inputs2,hidden)
            loss=loss_function(y_pred,y_train)
            final_losses.append(loss)
            if i%10==1:
                print("Epoch number: {} and the loss : {}".format(i,loss.item()))
            otimizer.zero_grad()
            loss.backward()
            otimizer.step()
print(time.time()-start_time)

RuntimeError: Expected hidden[0] size (2, 300, 512), got [2, 400, 512]

In [ ]:
## loss function
import matplotlib.pyplot as plt

plt.plot(range(epochs),final_losses)
plt.ylabel('Loss')
plt.xlabel('Epoch')
#### Prediction in X_test data
predictions=[]
with torch.no_grad():
    hidden = model.init_hidden(batch_size)
    for inputs1,inputs2, labels in val_loader:
        y_pred=model(inputs1,inputs2,hidden)
        predictions.append(y_pred.argmax().item())
        print(y_pred.argmax().item())

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,predictions)
cm
plt.figure(figsize=(10,6))
sns.heatmap(cm,annot=True)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,predictions)
cm

In [ ]:

from sklearn.metrics import accuracy_score

accuracy_score(y_test,predictions)

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_test,predictions)

In [ ]:
from sklearn.metrics import precision_score
precision_score(y_test,predictions)